In [1]:
!pip install transformers datasets huggingface_hub


In [2]:
import transformers
import datasets
import huggingface_hub


In [ ]:
from huggingface_hub import login

login(token="")

In [4]:
from datasets import load_dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the dataset
dataset = load_dataset("omi-health/medical-dialogue-to-soap-summary")

# Load the T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Preprocess the dialogues and SOAP notes for tokenization
def preprocess_and_tokenize(example):
    # Tokenize the dialogue (input text)
    input_ids = tokenizer(example["dialogue"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    # Tokenize the SOAP note (target text)
    labels = tokenizer(example["soap"], padding="max_length", truncation=True, max_length=512, return_tensors="pt").input_ids

    # Add labels to the input dictionary
    input_ids["labels"] = labels
    return input_ids

# Apply preprocessing and tokenization to the entire dataset
train_dataset = dataset['train'].map(preprocess_and_tokenize, batched=True)
val_dataset = dataset['validation'].map(preprocess_and_tokenize, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

train.json:   0%|          | 0.00/154M [00:00<?, ?B/s]

validation.json:   0%|          | 0.00/8.31M [00:00<?, ?B/s]

test.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/9250 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [5]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./results",            # Directory for saving the model
    num_train_epochs=3,                # Number of training epochs
    per_device_train_batch_size=16,     # Batch size for training
    per_device_eval_batch_size=16,      # Batch size for evaluation
    warmup_steps=500,                  # Warmup steps for learning rate scheduler
    weight_decay=0.01,                 # Weight decay to avoid overfitting
    logging_dir="./logs",              # Directory for logging
    logging_steps=10,                  # Log every 10 steps
    save_steps=500,                    # Save the model every 500 steps
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,                # Keep only the last 2 saved models
    load_best_model_at_end=True,       # Load the best model when finished
    report_to="none",
    fp16=True,                         # Enable mixed precision training for faster training
    metric_for_best_model="eval_loss",
)

In [6]:
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
)

In [8]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    callbacks=[early_stopping_callback],  # Add the early stopping callback here
)

# Fine-tune the model
trainer.train()

<ipython-input-8-1d1120dfcd62>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.606600,1.379356
2,1.530500,1.333092
3,1.519600,1.319480


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=1737, training_loss=1.5339265613632773, metrics={'train_runtime': 1210.5036, 'train_samples_per_second': 22.924, 'train_steps_per_second': 1.435, 'total_flos': 3755734990848000.0, 'train_loss': 1.5339265613632773, 'epoch': 3.0})

In [9]:
# Extract the first 100 dialogues from the test set
test_dialogues = dataset['test']['dialogue'][:100]


In [10]:
model.save_pretrained('./finetuned_t5_small')
tokenizer.save_pretrained('./finetuned_t5_small')


('./finetuned_t5_small/tokenizer_config.json',
 './finetuned_t5_small/special_tokens_map.json',
 './finetuned_t5_small/spiece.model',
 './finetuned_t5_small/added_tokens.json')

In [11]:
import torch


In [12]:
# Check if a GPU is available and use it
device = "cuda" if torch.cuda.is_available() else "cpu"

# Move the model to the appropriate device
model.to(device)

# Tokenize and move inputs to the same device as the model
inputs = tokenizer(test_dialogues, return_tensors="pt", padding=True, truncation=True, max_length=512)
inputs = {key: value.to(device) for key, value in inputs.items()}  # Move input tensors to the same device as the model

# Generate SOAP notes for these dialogues
outputs = model.generate(inputs['input_ids'], max_length=256, num_beams=5, early_stopping=True)

# Decode the generated token IDs into readable SOAP notes
generated_soap_notes = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Print the first few results to check
print(generated_soap_notes[:5])


["S: The patient reports experiencing painless blurry vision in the right eye for a week, intermittent fevers, headache, body aches, and a nonpruritic maculopapular rash on the lower legs for the past 6 months. The patient denies any other symptoms such as neck stiffness, nausea, vomiting, Raynaud's phenomenon, oral ulcerations, chest pain, shortness of breath, abdominal pain, or photosensitivity. O: Vital signs were normal. Physical examination revealed bilateral papilledema and optic nerve erythema in the right eye, right inferior nasal quadrant visual field defect, right afferent pupillary defect, normal muscle strength and reflexes, and intact sensation to light touch, pinprick, vibration, and proprioception. Laboratory findings include microcytic anemia with hemoglobin 11.6 gm/dL, hematocrit 35.3%, mean corpuscular volume of 76.9 fL, hyponatremia at 33 mm/hr, C-reactive protein (CRP) at 13.3 mg/L, and urinalysis did", 'S: The patient is a 7-year-old boy with a history of congenita

In [13]:
import csv

# Save the generated SOAP notes and dialogues to a CSV file
with open("generated_soap_notes_100.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Dialogue", "Generated SOAP Note"])  # Column headers
    for dialogue, soap_note in zip(test_dialogues, generated_soap_notes):
        writer.writerow([dialogue, soap_note])


In [14]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the fine-tuned model and tokenizer from the local directory on Colab
model = T5ForConditionalGeneration.from_pretrained('./finetuned_t5_small')
tokenizer = T5Tokenizer.from_pretrained('./finetuned_t5_small')


In [15]:
import pandas as pd

# Load the CSV file containing the generated SOAP notes
df = pd.read_csv('generated_soap_notes_100.csv')  # Path to your CSV file

# Extract the generated SOAP notes (assuming "Generated SOAP Note" column exists)
generated_soap_notes = df['Generated SOAP Note'].tolist()

# Print the first few SOAP notes to check
print(generated_soap_notes[:5])


["S: The patient reports experiencing painless blurry vision in the right eye for a week, intermittent fevers, headache, body aches, and a nonpruritic maculopapular rash on the lower legs for the past 6 months. The patient denies any other symptoms such as neck stiffness, nausea, vomiting, Raynaud's phenomenon, oral ulcerations, chest pain, shortness of breath, abdominal pain, or photosensitivity. O: Vital signs were normal. Physical examination revealed bilateral papilledema and optic nerve erythema in the right eye, right inferior nasal quadrant visual field defect, right afferent pupillary defect, normal muscle strength and reflexes, and intact sensation to light touch, pinprick, vibration, and proprioception. Laboratory findings include microcytic anemia with hemoglobin 11.6 gm/dL, hematocrit 35.3%, mean corpuscular volume of 76.9 fL, hyponatremia at 33 mm/hr, C-reactive protein (CRP) at 13.3 mg/L, and urinalysis did", 'S: The patient is a 7-year-old boy with a history of congenita

In [17]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00


In [19]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a609fd3fb8f7064b40699413c000163e2af2a90c8d94628d86dc320e931f1751
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [20]:
import torch
import evaluate
from datasets import load_dataset

# Load the ROUGE metric
rouge_metric = evaluate.load("rouge")


dataset = load_dataset("omi-health/medical-dialogue-to-soap-summary")

# Extract the ground truth SOAP notes from the test set
ground_truth_soap_notes = dataset['test']['soap'][:100]

# Evaluate the ROUGE score by comparing generated SOAP notes with ground truth
results = rouge_metric.compute(predictions=generated_soap_notes, references=ground_truth_soap_notes)

# Print the ROUGE results
print(results)


{'rouge1': np.float64(0.5116230006988383), 'rouge2': np.float64(0.3112252446438738), 'rougeL': np.float64(0.3737187407195166), 'rougeLsum': np.float64(0.43026811186402225)}


In [21]:
# Accessing specific ROUGE scores
rouge_1 = results["rouge1"]
rouge_2 = results["rouge2"]
rouge_l = results["rougeL"]

# Print each of the scores
print(f"ROUGE-1: {rouge_1}")
print(f"ROUGE-2: {rouge_2}")
print(f"ROUGE-L: {rouge_l}")


ROUGE-1: 0.5116230006988383
ROUGE-2: 0.3112252446438738
ROUGE-L: 0.3737187407195166
